<a href="https://colab.research.google.com/github/AkhilSai13/Automated-Stock-Trading-Bot/blob/main/LSTM_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as y
symbol_list = ["ADANIENT.NS", "ADANIPORTS.NS", "APOLLOHOSP.NS", "ASIANPAINT.NS", "AXISBANK.NS", "BAJAJ-AUTO.NS", "BAJAJFINSV.NS", "BHARTIARTL.NS", "BPCL.NS", "BRITANNIA.NS", "CIPLA.NS", "COALINDIA.NS", "DIVISLAB.NS", "DRREDDY.NS", "EICHERMOT.NS", "GRASIM.NS", "HCLTECH.NS", "HDFC.NS", "HDFCBANK.NS", "HDFCLIFE.NS", "HEROMOTOCO.NS", "HINDALCO.NS", "HINDUNILVR.NS", "ICICIBANK.NS", "INDUSINDBK.NS", "INFY.NS", "ITC.NS", "JSWSTEEL.NS", "KOTAKBANK.NS", "LT.NS", "M&M.NS", "NTPC.NS", "ONGC.NS", "POWERGRID.NS", "RELIANCE.NS", "SBILIFE.NS", "SBIN.NS", "SUNPHARMA.NS", "TATACONSUM.NS", "TATAMOTORS.NS", "TATASTEEL.NS", "TCS.NS", "TECHM.NS", "TITAN.NS", "UPL.NS", "WIPRO.NS"]
#symbol_list = ["TCS.NS", "WIPRO.NS", "ICICIBANK.NS", "NESTLEIND.NS", "RELIANCE.NS"]
datef = {}
rmse_dict = dict()
tp_d = {}
fp_d = {}
fn_d = {}
tn_d = {}

for j in symbol_list:
  dataset1 = y.download( j, start = '2012-01-02', end = '2023-03-31')
  temp1= y.download( j, start = '2012-01-02', end = '2019-12-31')

  tpg = fpg = fng = tng = 0
  dates = []
  rmset = []

  for i in range(5):

    dataset = dataset1[i::5]
    temp = temp1[i::5]
    #len(dataset)
    print(dataset.head())

    import matplotlib.pyplot as plt

    plt.figure(figsize=(20, 10))
    plt.plot(dataset['Close'])
    plt.xlabel('Date')
    plt.ylabel('Prices')
    plt.title('TCS Closing Prices')

    import math
    from sklearn.preprocessing import MinMaxScaler 
    import numpy as np


    close = dataset['Close']
    arr_close = close.values

    scale = MinMaxScaler(feature_range=(0,1))
    scaled = scale.fit_transform(arr_close.reshape(-1,1))
    #train_data_len = math.ceil(len(arr_close)*0.8)
    #train_data_len = math.ceil(len(arr_close)-14)
    train_data_len = len(temp)
    train_data = scaled[0:train_data_len, :]

    print(len(arr_close), train_data_len)
    x_train = []
    y_train = []

    for i in range(60, len(train_data)):
        x_train.append(train_data[i-60:i, 0])
        y_train.append(train_data[i, 0])
        
    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


    test_data = scaled[train_data_len-60: , : ]
    x_test = []
    y_test = arr_close[train_data_len:]

    for i in range(60, len(test_data)):
      x_test.append(test_data[i-60:i, 0])

    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    #print(x_test[0])

    '''
    from tensorflow import keras
    from tensorflow.keras import layers

    model = keras.Sequential()
    model.add(layers.LSTM(100, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(layers.LSTM(100, return_sequences=False))
    model.add(layers.Dense(25))
    model.add(layers.Dense(1))
    model.summary()
    '''

    from tensorflow import keras
    from tensorflow.keras import layers

    model = keras.Sequential()
    model.add(layers.LSTM(100, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(layers.Dropout(rate = 0.2))
    #model.add(layers.LSTM(100, return_sequences=True))
    #model.add(layers.Dropout(rate = 0.2))
    #model.add(layers.LSTM(100, return_sequences=True))
    #model.add(layers.Dropout(rate = 0.2))
    model.add(layers.LSTM(100, return_sequences=False))
    model.add(layers.Dropout(rate = 0.2))
    model.add(layers.Dense(25))
    model.add(layers.Dense(1))
    model.summary()

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, batch_size= 32, epochs=100)

    predictions = model.predict(x_test)
    #print(predictions)
    predictions = scale.inverse_transform(predictions)

    rmse = np.sqrt(np.mean(predictions - y_test)**2)
    print(rmse)
    rmset.append(rmse)

    #print(predictions)

    data = dataset.filter(['Close'])
    train = data[:train_data_len]
    validation = data[train_data_len:]
    validation['Predictions'] = predictions
    plt.figure(figsize=(20,10))
    plt.title('Model')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.plot(train)
    plt.plot(validation[['Close', 'Predictions']])
    plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
    plt.show()

    a = []
    tp = 0
    tn = 0
    fp = 0
    fn = 0

    import pandas
    from datetime import date, timedelta

    tdata = dataset[len(temp)-1:]
    drange = tdata.reset_index()['Date']
    #dates2 = dict()
    d = str(drange[0].strftime('%Y-%m-%d'))

    if (predictions[0][0] > y_train[-1] and y_test[0] > y_train[-1]):
      tp+=1
      dates.append(d)
      #dates2[d] = 1
    elif (predictions[0][0] > y_train[-1] and y_test[0] < y_train[-1]):
      fp+=1
      dates.append(d)
      #dates2[d] = 1
    elif (predictions[0][0] < y_train[-1] and y_test[0] > y_train[-1]):
      fn+=1
      #dates2[d] = 0
    else:
      tn+=1
      #dates2[d] = 0

    for i in range(1, len(predictions)):
      d = str(drange[i].strftime('%Y-%m-%d'))
      if (predictions[i][0] > y_test[i-1] and y_test[i] > y_test[i-1]):
        tp+=1
        dates.append(d)
      elif (predictions[i][0] > y_test[i-1] and y_test[i] < y_test[i-1]):
        fp+=1
        dates.append(d)
      elif (predictions[i][0] < y_test[i-1] and y_test[i] > y_test[i-1]):
        fn+=1
      else:
        tn+=1
      

    for i in range(len(predictions)):
      a.append(abs(predictions[i][0] - y_test[i])/y_test[i]*100)
    print(max(a), min(a), sum(a)/len(a))
    print(tp, fp, fn, tn)
    tpg = tpg + tp
    fpg = fpg + fp
    fng = fng + fn
    tng = tng + tn

    #print(tp+tn+fp+fn)
    #print(len(y_test))
    print((tp+tn)/(tp+tn+fn+fp)*100) # accuracy
    #print((tp)/(tp+fn)*100) # sensitivity
    #print((tp)/(tp+fp)*100) # presicion
    #print((tn)/(fp+tn)*100) # specificity
    #print( (1 - (tn)/(fp+tn))*100) # 1 - specificity
    #print(predictions, y_test)
  print(tpg, tng, fng, fpg)
  dates.sort()
  #print(len(dates))
  datef[j] = dates
  rmse_dict[j] = rmset
  tp_d[j] = tpg
  fp_d[j] = fpg
  tn_d[j] = tng
  fn_d[j] = fng
  #print(dates)

#print(datef)


In [ ]:
print(rmse_dict)
for i in rmse_dict:
  rmse_dict[i] = sum(rmse_dict[i])/len(rmse_dict[i])

print(rmse_dict)


In [ ]:
print(datef)

In [ ]:
print(tp_d)

In [ ]:
print(tn_d)

In [ ]:
print(fp_d)

In [ ]:
print(fn_d)